In [1]:
import tensorflow as tf
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, Dropout,Embedding, GRU, BatchNormalization, Input, MultiHeadAttention, Attention
from tensorflow.keras.layers import Conv1D, Conv1DTranspose
from tensorflow.keras.activations import relu, softmax, gelu
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.losses import mean_squared_error, categorical_crossentropy, sparse_categorical_crossentropy
from tensorflow.keras.layers import Concatenate, concatenate
from tensorflow.keras.callbacks import ReduceLROnPlateau, LearningRateScheduler
import numpy as np

In [44]:
class self_attention(Layer):
    def __init__(self, heads = 5, causal = True, dropout = 0.2):
        super().__init__()
        self.causal = causal 

        self.dropout = Dropout(dropout)
    def build(self, input_shape, **kwargs):
        super().__init__(**kwargs)
        
        input_shape = input_shape[0]
        shape = input_shape[-2], input_shape[-2]
        initializer = tf.keras.initializers.Orthogonal()  #### we in particular ortogonal initialization, in the case that it is needed it will train it!
        initial_value = initializer(shape = shape)
        self.kernel = tf.Variable(initial_value = initial_value, trainable = True)
        if self.causal: ### this part is used to kill attention of future to past, 
            minf = -tf.constant(20000.0)  ### take this dude to kill softmax maybe a little bit smaller.
            mask = tf.fill(shape, minf)
            self.upper_m = minf - tf.linalg.band_part(mask, num_lower = -1, num_upper = 0)
            
    @tf.function()        
    def call(self, inputs, training = None):
        if training:
            inputs = self.dropout(inputs, training) ### dropout is applied in the begining of the layer
        input_1 = tf.matmul(self.kernel, inputs[0])
        input_2 = tf.matmul(self.kernel, inputs[1])
        att_scores = tf.matmul(input_1, input_2, transpose_b = True)  ### we get the attention scores
        d_k = (input_1.shape[-2])**0.5 ####normalizing factor is here
        if self.causal:
            return tf.nn.softmax((att_scores+self.upper_m)/d_k) @ input_1
        
        return  tf.nn.softmax(att_scores/d_k) @ input_1 

In [52]:
a = self_attention()
np.random.seed(1)
x = np.random.randn(1,3,2)
a([x,x]),x

(<tf.Tensor: shape=(1, 3, 2), dtype=float32, numpy=
 array([[[ 0.9132909 ,  1.0458729 ],
         [ 1.473622  , -1.0643605 ],
         [-0.45192552,  1.8578761 ]]], dtype=float32)>,
 array([[[ 1.62434536, -0.61175641],
         [-0.52817175, -1.07296862],
         [ 0.86540763, -2.3015387 ]]]))

In [39]:
np.random.seed(1)
np.random.randn()

1.6243453636632417